# 03 — Modeling & Evaluation (Early Default)
#
# **Objective**
# Train and evaluate predictive models for `early_default` using a realistic temporal split.
#
# **Models**
# 1) Logistic Regression (interpretable baseline)
# 2) XGBoost (nonlinear model with class-imbalance handling)
#
# **Evaluation**
# - ROC-AUC (ranking ability)
# - PR-AUC (rare-event performance)
# - Threshold policy (recall / review-rate tradeoffs)
# - Calibration check (are probabilities meaningful as PDs?)
# - Lift (risk concentration in high-score segments)


In [1]:
import pandas as pd
import numpy as np

# 1) Load Processed Dataset (from Notebook 02)
# This dataset is already:
# - target-defined (`early_default`)
# - filtered for observability
# - imputed
# - one-hot encoded
#
# `issue_d` is retained only for temporal splitting, not as a predictive feature.


In [2]:
df_model_encoded = pd.read_csv(
    "../data/processed/early_default_modeling_dataset.csv",
    parse_dates=["issue_d"]
)

# 2) Temporal Train/Test Split
# To simulate real deployment:
# - Train on older loans (pre-2015)
# - Test on newer loans (2015+)
# This avoids look-ahead bias and provides realistic generalization testing.

In [3]:
train = df_model_encoded[df_model_encoded['issue_d'] < '2015-01-01']
test  = df_model_encoded[df_model_encoded['issue_d'] >= '2015-01-01']


# 3) Define Features (X) and Target (y)
# - Drop `early_default` (target) and `issue_d` (split key, not a feature).


In [4]:
X_train = train.drop(columns=['early_default', 'issue_d'])
y_train = train['early_default']

X_test = test.drop(columns=['early_default', 'issue_d'])
y_test = test['early_default']

# 4) Sanity Check: Default Rate Drift
# We expect some drift over time in real credit portfolios. This helps contextualize performance.


In [5]:
print("Train default rate:", y_train.mean())
print("Test default rate:", y_test.mean())


Train default rate: 0.018583582566276607
Test default rate: 0.023350342115899526


# 5) Baseline Model: Logistic Regression
# Logistic regression is a strong, interpretable benchmark. We:
# - standardize numeric features
# - handle imbalance using `class_weight='balanced'`


In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'  # important due to imbalance
)

log_model.fit(X_train_scaled, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

# Logistic Evaluation
# ROC-AUC measures ranking ability; PR-AUC is more informative for rare events (early default).


In [8]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

y_proba_log = log_model.predict_proba(X_test_scaled)[:,1]

roc_auc_log = roc_auc_score(y_test, y_proba_log)
print("Logistic ROC-AUC:", roc_auc_log)

precision, recall, _ = precision_recall_curve(y_test, y_proba_log)
pr_auc_log = auc(recall, precision)
print("Logistic PR-AUC:", pr_auc_log)


Logistic ROC-AUC: 0.7188400266918514
Logistic PR-AUC: 0.05862826280295273


# 6) Advanced Model: XGBoost
# XGBoost can capture nonlinearities and interactions. For class imbalance, we use:
# `scale_pos_weight = #neg / #pos` (preferred over SMOTE for large datasets).


In [9]:
neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos
print("scale_pos_weight:", scale_pos_weight)


scale_pos_weight: 52.8109375


# XGBoost Feature-Name Cleaning
# XGBoost is strict about feature names and may error if column names contain special characters
# (e.g., "<", "]", "["). We sanitize names to avoid training errors.


In [10]:
bad_cols = [c for c in X_train.columns if any(ch in c for ch in ['[', ']', '<'])]

bad_cols[:30], len(bad_cols)

(['emp_length_< 1 year'], 1)

In [11]:
import re

def clean_feature_names(df):
    df = df.copy()
    df.columns = [re.sub(r'[^A-Za-z0-9_]+', '_', str(c)) for c in df.columns]
    return df

X_train_xgb = clean_feature_names(X_train)
X_test_xgb  = clean_feature_names(X_test)

# Extra safety: ensure same columns/order
X_test_xgb = X_test_xgb.reindex(columns=X_train_xgb.columns, fill_value=0)


# Train XGBoost

In [12]:
from xgboost import XGBClassifier

neg, pos = np.bincount(y_train)
scale_pos_weight = neg / pos

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss'
)

xgb_model.fit(X_train_xgb, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

# XGBoost Evaluation

In [13]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

y_proba_xgb = xgb_model.predict_proba(X_test_xgb)[:, 1]

roc_auc_xgb = roc_auc_score(y_test, y_proba_xgb)
precision, recall, _ = precision_recall_curve(y_test, y_proba_xgb)
pr_auc_xgb = auc(recall, precision)

print("XGBoost ROC-AUC:", roc_auc_xgb)
print("XGBoost PR-AUC:", pr_auc_xgb)


XGBoost ROC-AUC: 0.7187190802051603
XGBoost PR-AUC: 0.06009614764149687


# We save:
# - scaler (needed for logistic)
# - logistic model
# - xgboost model
#
# Note: XGBoost training used `X_train_xgb` (cleaned feature names). When scoring new data, you must apply
# the same `clean_feature_names()` and align columns.


In [14]:
import joblib

joblib.dump(scaler, "../models/scaler.joblib")
joblib.dump(log_model, "../models/log_model.joblib")
joblib.dump(xgb_model, "../models/xgb_model.joblib")

print("Saved: models/scaler.joblib, models/log_model.joblib, models/xgb_model.joblib")

Saved: models/scaler.joblib, models/log_model.joblib, models/xgb_model.joblib


# 7) Threshold Optimization (Model Operating Point)
# For rare events, threshold=0.5 is often not optimal. We evaluate thresholds from 0.01 to 0.50.
# F1 is included for reference, but operational thresholding in credit often prioritizes recall and capacity.


In [15]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

thresholds = np.arange(0.01, 0.51, 0.01)

precisions = []
recalls = []
f1s = []

for t in thresholds:
    y_pred = (y_proba_xgb > t).astype(int)
    precisions.append(precision_score(y_test, y_pred))
    recalls.append(recall_score(y_test, y_pred))
    f1s.append(f1_score(y_test, y_pred))

best_idx = np.argmax(f1s)
best_threshold = thresholds[best_idx]

print("Best threshold:", best_threshold)
print("Precision:", precisions[best_idx])
print("Recall:", recalls[best_idx])
print("F1:", f1s[best_idx])
    

Best threshold: 0.5
Precision: 0.04894054263547346
Recall: 0.5729770607511974
F1: 0.09017853206964954


In [16]:
results = pd.DataFrame({
    "threshold": thresholds,
    "precision": precisions,
    "recall": recalls,
    "f1": f1s
}).sort_values("f1", ascending=False)

results.head(10)


,threshold,precision,recall,f1
49,0.50,0.048941,0.572977,0.090179
48,0.49,0.047998,0.590875,0.088784
47,0.48,0.047018,0.607596,0.087282
46,0.47,0.046098,0.624289,0.085856
45,0.46,0.045172,0.640142,0.084390
44,0.45,0.044277,0.655435,0.082950
43,0.44,0.043432,0.670672,0.081580
42,0.43,0.042564,0.684984,0.080148
41,0.42,0.041805,0.700417,0.078901
40,0.41,0.041101,0.715850,0.077739


# 8) Policy Threshold: Target Recall (Early Warning)
# Early warning systems often prioritize catching defaults (high recall).
# We choose the threshold that achieves at least 80% recall with the best available precision.


In [17]:
target_recall = 0.80
eligible = results[results["recall"] >= target_recall]
best_policy = eligible.sort_values("precision", ascending=False).head(1)
best_policy

,threshold,precision,recall,f1
33,0.34,0.036374,0.810912,0.069625


In [18]:
policy_threshold = float(best_policy["threshold"].iloc[0])
print("Policy threshold (>=80% recall):", policy_threshold)

Policy threshold (>=80% recall): 0.34


In [20]:
# Save the chosen operating threshold so the same decision rule can be reused later.
joblib.dump({"policy_threshold": policy_threshold}, "../models/policy_threshold.joblib")
print("Saved: models/policy_threshold.joblib")

Saved: models/policy_threshold.joblib


In [21]:
y_proba_xgb.min(), y_proba_xgb.max(), y_proba_xgb.mean()


(np.float32(0.008400743), np.float32(0.9270754), np.float32(0.37171197))

In [22]:
import pandas as pd

# Calibration table (deciles)
cal = pd.DataFrame({"p": y_proba_xgb, "y": y_test.values})
cal["bin"] = pd.qcut(cal["p"], 10, duplicates="drop")

cal_summary = cal.groupby("bin").agg(
    avg_pred=("p","mean"),
    event_rate=("y","mean"),
    count=("y","size")
).reset_index()

cal_summary


C:\Users\bdgsg\AppData\Local\Temp\ipykernel_25544\178686767.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  cal_summary = cal.groupby("bin").agg(


,bin,avg_pred,event_rate,count
0,"(0.0073999999999999995, 0.128]",0.090003,0.003643,152902
1,"(0.128, 0.19]",0.159579,0.006422,152901
2,"(0.19, 0.243]",0.216755,0.009810,152901
3,"(0.243, 0.295]",0.268897,0.012099,152902
4,"(0.295, 0.352]",0.323195,0.015638,152901
5,"(0.352, 0.414]",0.382835,0.020268,152901
6,"(0.414, 0.481]",0.447318,0.024297,152902
7,"(0.481, 0.554]",0.517102,0.031380,152901
8,"(0.554, 0.642]",0.596156,0.040647,152901
9,"(0.642, 0.927]",0.715281,0.069299,152902


# 10) Performance at Policy Threshold
# We report:
# - review/flag rate
# - confusion matrix
# - classification report
# This translates model scores into an operational decision rule.

In [23]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions at policy threshold
y_pred_policy = (y_proba_xgb >= policy_threshold).astype(int)

# Flag rate (review rate)
flag_rate = y_pred_policy.mean()

print("Policy threshold:", policy_threshold)
print("Flag / review rate:", flag_rate)

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_policy)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n", classification_report(y_test, y_pred_policy))


Policy threshold: 0.34
Flag / review rate: 0.5205681569952924
Confusion Matrix:
 [[726307 767004]
 [  6751  28952]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.49      0.65   1493311
           1       0.04      0.81      0.07     35703

    accuracy                           0.49   1529014
   macro avg       0.51      0.65      0.36   1529014
weighted avg       0.97      0.49      0.64   1529014



# 11) Lift (Risk Concentration)
# Lift compares the default rate in the highest-risk segment to the overall portfolio default rate.


In [24]:
overall_rate = y_test.mean()

# Use your calibration table (last bin = highest risk decile)
top_decile_rate = cal_summary.iloc[-1]["event_rate"]

lift = top_decile_rate / overall_rate

print("Overall default rate:", overall_rate)
print("Top decile default rate:", top_decile_rate)
print("Top decile lift:", lift)


Overall default rate: 0.023350342115899526
Top decile default rate: 0.06929928974114138
Top decile lift: 2.967806184473617


# 12) Review-Capacity Scenarios
# Instead of choosing threshold purely by F1, we evaluate thresholds based on review capacity:
# - top 10%, 20%, 30% flagged
# This is a realistic operational framing for early warning systems.


In [25]:
for target_flag_rate in [0.10, 0.20, 0.30]:
    thresh = np.percentile(y_proba_xgb, 100*(1-target_flag_rate))
    y_pred_tmp = (y_proba_xgb >= thresh).astype(int)
    recall_tmp = recall_score(y_test, y_pred_tmp)
    precision_tmp = precision_score(y_test, y_pred_tmp)
    print(f"\nTarget flag rate: {target_flag_rate}")
    print("Threshold:", round(thresh,3))
    print("Precision:", round(precision_tmp,4))
    print("Recall:", round(recall_tmp,4))



Target flag rate: 0.1
Threshold: 0.642
Precision: 0.0693
Recall: 0.2968

Target flag rate: 0.2
Threshold: 0.554
Precision: 0.055
Recall: 0.4709

Target flag rate: 0.3
Threshold: 0.481
Precision: 0.0471
Recall: 0.6052
